In [13]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import  models, transforms,datasets
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import time
import math
import numpy as np
import os,cv2
from torch.utils.data import Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from PIL import Image
import torch.nn.init as init


batch_size = 4
num_classes = 4
learning_rate = 0.001
momentum = 0.9
img_size = 224
# 定义一个类，需要创建模型的时候，就实例化一个对象

class VehicleColorRecognitionModel(nn.Module):
    def __init__(self,Load_VIS_URL=None):
        super(VehicleColorRecognitionModel,self).__init__()
        
        # ===============================  top ================================
        # first top convolution layer   
        self.top_conv1 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(3, 48, kernel_size=(11,11), stride=(4,4)),
            nn.ReLU(),
            nn.BatchNorm2d(48),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
       
        
        # first top convolution layer    after split
        self.top_top_conv2 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(24, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.top_bot_conv2 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(24, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        
        #  need a concat
        
        # after concat  
        self.top_conv3 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(128, 192, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU()
        )
        
        # fourth top convolution layer
        # split feature map by half
        self.top_top_conv4 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 96, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU()
        )
        
        self.top_bot_conv4 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 96, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU()
        )
        
        
        # fifth top convolution layer
        self.top_top_conv5 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.top_bot_conv5 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

#        # ===============================  bottom ================================
    
           
#         # first bottom convolution layer   
        self.bottom_conv1 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(3, 48, kernel_size=(11,11), stride=(4,4)),
            nn.ReLU(),
            nn.BatchNorm2d(48),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
       
        
        # first top convolution layer    after split
        self.bottom_top_conv2 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(24, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.bottom_bot_conv2 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(24, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        
        #  need a concat
        
        # after concat  
        self.bottom_conv3 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(128, 192, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU()
        )
        
        # fourth top convolution layer
        # split feature map by half
        self.bottom_top_conv4 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 96, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU()
        )
        
        self.bottom_bot_conv4 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 96, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU()
        )
        
        
        # fifth top convolution layer
        self.bottom_top_conv5 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.bottom_bot_conv5 = nn.Sequential(
            # 1-1 conv layer
            nn.Conv2d(96, 64, kernel_size=(3,3), stride=(1,1),padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        # Fully-connected layer
        self.classifier = nn.Sequential(
            nn.Linear(5*5*64*4, 4096), 
            nn.ReLU(), 
            nn.Dropout(0.7),
            nn.Linear(4096, 4096),
            nn.ReLU(), 
            nn.Dropout(0.6),
            nn.Linear(4096, num_classes)
        )                
        
    def forward(self,x):
        #print(x.shape)
        x_top = self.top_conv1(x)
        #print(x_top.shape)
                
        x_top_conv = torch.split(x_top, 24, 1)
        
        x_top_top_conv2 = self.top_top_conv2(x_top_conv[0])
        x_top_bot_conv2 = self.top_bot_conv2(x_top_conv[1])
        
        x_top_cat1 = torch.cat([x_top_top_conv2,x_top_bot_conv2],1)
        
        x_top_conv3 = self.top_conv3(x_top_cat1)
        
        x_top_conv3 = torch.split(x_top_conv3, 96, 1)
        
        x_top_top_conv4 = self.top_top_conv4(x_top_conv3[0])
        x_top_bot_conv4 = self.top_bot_conv4(x_top_conv3[1])
        
        x_top_top_conv5 = self.top_top_conv5(x_top_top_conv4)
        x_top_bot_conv5 = self.top_bot_conv5(x_top_bot_conv4)
        
        x_bottom = self.bottom_conv1(x)
        
        x_bottom_conv = torch.split(x_bottom, 24, 1)
        
        x_bottom_top_conv2 = self.bottom_top_conv2(x_bottom_conv[0])
        x_bottom_bot_conv2 = self.bottom_bot_conv2(x_bottom_conv[1])
        
        x_bottom_cat1 = torch.cat([x_bottom_top_conv2,x_bottom_bot_conv2],1)
        
        x_bottom_conv3 = self.bottom_conv3(x_bottom_cat1)
        
        x_bottom_conv3 = torch.split(x_bottom_conv3, 96, 1)
        
        x_bottom_top_conv4 = self.bottom_top_conv4(x_bottom_conv3[0])
        x_bottom_bot_conv4 = self.bottom_bot_conv4(x_bottom_conv3[1])
        
        x_bottom_top_conv5 = self.bottom_top_conv5(x_bottom_top_conv4)
        x_bottom_bot_conv5 = self.bottom_bot_conv5(x_bottom_bot_conv4)
        
        x_cat = torch.cat([x_top_top_conv5,x_top_bot_conv5,x_bottom_top_conv5,x_bottom_bot_conv5],1)
        
        
        flatten = x_cat.view(x_cat.size(0), -1)
        
        output = self.classifier(flatten)
        
        #output = F.softmax(output)
        
        
        return output

# Weights initialisation
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

net = VehicleColorRecognitionModel()
net.apply(weights_init)


VehicleColorRecognitionModel(
  (top_conv1): Sequential(
    (0): Conv2d(3, 48, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (top_top_conv2): Sequential(
    (0): Conv2d(24, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (top_bot_conv2): Sequential(
    (0): Conv2d(24, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (top_conv3): Sequential(
    (0): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padd

In [2]:
# import torch
# from torch.utils import data

# class Dataset(data.Dataset):
#     'Characterizes a dataset for PyTorch'
#     def __init__(self, list_IDs, labels):
#         'Initialization'
#         self.labels = labels
#         self.list_IDs = list_IDs

#     def __len__(self):
#         'Denotes the total number of samples'
#         return len(self.list_IDs)

#     def __getitem__(self, index):
#         'Generates one sample of data'
#         # Select sample
#         ID = self.list_IDs[index]

#         # Load data and get label
#         X = torch.load('data/' + ID + '.png')
#         y = self.labels[ID]

#         return X, y

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)



In [4]:
# Generate fake data

# inputs = torch.rand(batch_size,3,img_size,img_size)
# labels = torch.from_numpy(np.random.choice(num_classes,batch_size))
# data_loader = [(inputs, labels)]

In [5]:
# Image preprocessing
# doesnt work
# image_path = "Path to your dataset"

# def loadImages(path):
#     # Put files into lists and return them as one list of size 4
#     image_files = sorted([os.path.join(path, 'training', file)
#          for file in os.listdir(path + "/training") if      file.endswith('.png')])
 
#     return image_files

# # Preprocessing
# def processing(data):
#     # loading image
#     # Getting 3 images to work with 
#     img = [cv2.imread(i, cv2.IMREAD_COLOR) for i in data]
#     print(img)
#     print('Original size',img.shape)
#     # --------------------------------
#     # setting dim of the resize
#     height = 220
#     width = 220
#     dim = (width, height)
#     res_img = []
#     for i in range(len(img)):
#         res = cv2.resize(img[i], dim, interpolation=cv2.INTER_LINEAR)
#         res_img.append(res)


In [6]:
# raw_img = loadImages(path="./data")
# raw_img
# print(cv2.imread('/Users/weitat/Projects/fbhackathon/Pytorch_VehicleColorRecognition/data/training/abc.png'))

In [7]:
# def resize_tensor(input_tensors, h, w):
#     final_output = None
#     batch_size, channel, height, width = input_tensors.shape
#     #print(input_tensors.shape)
#     input_tensors = torch.squeeze(input_tensors, 1)
#     #print(input_tensors.shape)
#     print("hello")
#     for img in input_tensors:
#         print(img.shape)
#         img_PIL = transforms.ToPILImage()(img)
#         img_PIL = torchvision.transforms.Resize([h,w])(img_PIL)
#         img_PIL = torchvision.transforms.ToTensor()(img_PIL)
#         print(img_PIL.shape)
#         if final_output is None:
#             final_output = img_PIL
#         else:
#             final_output = torch.cat((final_output, img_PIL), 0)
# #     final_output = torch.unsqueeze(final_output, 1)
#     return final_output

In [51]:
transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.CenterCrop(224),
        transforms.ToTensor() 
     ],
)

# Load Data from image folder
train_path='data/train/'

train_dataset = torchvision.datasets.ImageFolder(
    root=train_path,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,
    num_workers=0,
    shuffle=True
)

# Load Data from image folder
test_path='data/test/'

test_dataset = torchvision.datasets.ImageFolder(
    root=test_path,
    transform=transform
)


test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=4,
    num_workers=0,
    shuffle=True
)
# classes = ('Dancer', 'Undressed_Pink')
# partition = {'train': ['00001'], 'validation': ['00002']}
# labels = {'00001.png':0, '00002.png':1}


# training_set = Dataset(partition['train'], labels)
# training_generator = data.DataLoader(training_set, batch_size=batch_size,
#                                     shuffle=True, num_workers=1)

# validation_set = Dataset(partition['validation'], labels)
# validation_generator = data.DataLoader(validation_set, batch_size=batch_size,
#                                     shuffle=True, num_workers=1)

In [52]:
train_dataset.classes

['images_#303765', 'images_#97264f', 'images_#aa4734', 'images_#ac3122']

In [9]:
# import matplotlib.pyplot as plt
# it = iter(train_loader)
# # it.next()
# print(it.dataset)
# print(it.next())
# img_test = it.next()[0]
# print(img_test.shape)
# plt.imshow(img_test[0].permute(1,2,0))
# test = F.interpolate(img_test, 224)
# plt.figure()
# plt.imshow(  test[0].permute(1, 2, 0)  )


In [10]:
for epoch in range(100):  # loop over the dataset multiple times
    print("Epoch: ",epoch)
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
#         print(inputs.shape)
#         inputs=resize_tensor(inputs,224,224)
#         print(inputs.shape)
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))    
        
        correct = 0
        total = 0
        for data in train_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 train images: %d %%' % (
        100 * correct / total))    
print('Finished Training')

Epoch:  0
[1,    20] loss: 0.014
[1,    40] loss: 0.015
[1,    60] loss: 0.015
Accuracy of the network on the 10000 test images: 21 %
Accuracy of the network on the 10000 train images: 25 %
Epoch:  1
[2,    20] loss: 0.015
[2,    40] loss: 0.015
[2,    60] loss: 0.014
Accuracy of the network on the 10000 test images: 31 %
Accuracy of the network on the 10000 train images: 33 %
Epoch:  2
[3,    20] loss: 0.013
[3,    40] loss: 0.012
[3,    60] loss: 0.013
Accuracy of the network on the 10000 test images: 37 %
Accuracy of the network on the 10000 train images: 44 %
Epoch:  3
[4,    20] loss: 0.011
[4,    40] loss: 0.009
[4,    60] loss: 0.010
Accuracy of the network on the 10000 test images: 43 %
Accuracy of the network on the 10000 train images: 52 %
Epoch:  4
[5,    20] loss: 0.010
[5,    40] loss: 0.009
[5,    60] loss: 0.010
Accuracy of the network on the 10000 test images: 50 %
Accuracy of the network on the 10000 train images: 55 %
Epoch:  5
[6,    20] loss: 0.011
[6,    40] loss: 

Accuracy of the network on the 10000 test images: 75 %
Accuracy of the network on the 10000 train images: 87 %
Epoch:  43
[44,    20] loss: 0.004
[44,    40] loss: 0.003
[44,    60] loss: 0.003
Accuracy of the network on the 10000 test images: 75 %
Accuracy of the network on the 10000 train images: 83 %
Epoch:  44
[45,    20] loss: 0.002
[45,    40] loss: 0.002
[45,    60] loss: 0.003
Accuracy of the network on the 10000 test images: 87 %
Accuracy of the network on the 10000 train images: 97 %
Epoch:  45
[46,    20] loss: 0.002
[46,    40] loss: 0.001
[46,    60] loss: 0.002
Accuracy of the network on the 10000 test images: 89 %
Accuracy of the network on the 10000 train images: 98 %
Epoch:  46
[47,    20] loss: 0.003
[47,    40] loss: 0.004
[47,    60] loss: 0.003
Accuracy of the network on the 10000 test images: 84 %
Accuracy of the network on the 10000 train images: 93 %
Epoch:  47
[48,    20] loss: 0.001
[48,    40] loss: 0.001
[48,    60] loss: 0.001
Accuracy of the network on the

Accuracy of the network on the 10000 train images: 100 %
Epoch:  85
[86,    20] loss: 0.000
[86,    40] loss: 0.000
[86,    60] loss: 0.000
Accuracy of the network on the 10000 test images: 92 %
Accuracy of the network on the 10000 train images: 99 %
Epoch:  86
[87,    20] loss: 0.000
[87,    40] loss: 0.000
[87,    60] loss: 0.000
Accuracy of the network on the 10000 test images: 93 %
Accuracy of the network on the 10000 train images: 100 %
Epoch:  87
[88,    20] loss: 0.000
[88,    40] loss: 0.000
[88,    60] loss: 0.000
Accuracy of the network on the 10000 test images: 96 %
Accuracy of the network on the 10000 train images: 100 %
Epoch:  88
[89,    20] loss: 0.000
[89,    40] loss: 0.000
[89,    60] loss: 0.000
Accuracy of the network on the 10000 test images: 94 %
Accuracy of the network on the 10000 train images: 100 %
Epoch:  89
[90,    20] loss: 0.000
[90,    40] loss: 0.000
[90,    60] loss: 0.000
Accuracy of the network on the 10000 test images: 94 %
Accuracy of the network on

In [94]:
PATH = './lipstick_net_test1.pth'
torch.save(net.state_dict(), PATH)

In [87]:

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 96 %


In [92]:
predicted

tensor([3, 0])

In [ ]:
load_model = VehicleColorRecognitionModel()
load_model.zload_state_dict(torch.load("./trained_parameter/BEST_lipstick_net.pth"))



In [80]:
classes = ['images_#303765', 'images_#97264f', 'images_#aa4734', 'images_#ac3122']

# Load Model and 1 forward passs
load_model = VehicleColorRecognitionModel()
load_model.load_state_dict(torch.load("./trained_parameter/BEST_lipstick_net.pth"))
net=load_model 
# Load Data from image folder
input_path = "../input/"

input_dataset = torchvision.datasets.ImageFolder(
    root=train_path,
    transform=transform
)

loader = torch.utils.data.DataLoader(
    input_dataset,
    batch_size=1,
    num_workers=0,
    shuffle=True
)

with torch.no_grad():
    iter_ = iter(loader)
    img_tensor = iter_.next()[0]
    outputs = load_model(img_tensor)
    _, predicted = torch.max(outputs, 1)
    print(classes[predicted])
    

images_#97264f


In [44]:
import json

data = {}
data['lipstick'] = []

url = 'https://www.maybelline.com/lip-makeup/lipstick/superstay-matte-ink-city-edition-liquid-lipstick-makeup'
if(predicted == 0):
    data['lipstick'].append({
        'color': '#303765',
        'name': 'Explorer',
        'url': url
    })
elif(predicted == 1):
    data['lipstick'].append({
    'color': '#97264F',
    'name': 'Artist',
    'url': url
    })
elif(predicted == 2):
    data['lipstick'].append({
    'color': '#AA4734',
    'name': 'Globetrotter',
    'url': url
    })
elif(predicted == 3):
    data['lipstick'].append({
    'color': '#AC3122',
    'name': 'Dancer',
    'url': url
    })
    
with open('../output/output.json', 'w') as outfile:
    json.dump(data, outfile)